In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from kk import dggs
from kk.dggs.io import save_png, dump_text, load_polygons
from kk import uitools

dg = dggs.DGGS()

In [ ]:
poly, src_crs = load_polygons('/Users/kirill/wk/dggs/data/act_aea.shp')
poly = poly[0]
display(poly, src_crs)

In [ ]:
im = dggs.shape_to_mask(poly, src_crs, 12, align=1)

im

In [ ]:
ff = dg.mk_display_helper()

fig = plt.figure(figsize=(6,6))
ax = fig.add_axes((0,0,1,1))
im_, extent = ff(im.roi.addr, im.value)
ax.imshow(im_, extent=extent)
uitools.plot_roi(im.roi, 'c-', ax=ax, alpha=0.6, linewidth=5);

In [ ]:
aa = dggs.mask_to_addresses(im, dg=dg)
print('\n'.join(aa[:5] + ['...'] + aa[-5:]))

In [ ]:
save_png('mm-mask-full-res.png', im.value, binary=True)
dump_text(aa, 'mm-addresses.txt')
dump_text(aa, 'mm-addresses.txt.gz')
dump_text(aa, 'mm-addresses.txt.xz')

!ls -lh mm-*

-------------------------------------------